# Compute effectiveness maps for evaluated methods

## Read and preprocess data

In [ ]:
import random
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_csv("_experiments/top_methods/all_results.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

In [ ]:
ss_methods = df["selection_metric"].unique()
protocols = sorted(df["protocol"].unique(), reverse=True)  # this is only to keep the order: OR and AND

# in some experiments mi_value was saved tih too big precision (differences 
# in 15th floating place) hence we need to round them 
df["mi_value"] = df["mi_value"].round(1)
mi_vals = df["mi_value"].unique()

# permute networks so that they're ordered as following:
nets = ["arxiv", "timik1q2009"]
assert set(df["network"].unique()) == set(nets)

# set up a path where results are saved in
workdir = Path("_results")
workdir.mkdir(exist_ok=True)

ss_methods_abbrv_map ={
    "cbim": "cbim",
    "cim": "cim",
    'degree_centrality': "deg-c",
    'degree_centrality_discount': "deg-c-d",
    'greedy': "greedy",
    'k_shell': "k-sh",
    'k_shell_mln': "k-sh-m",
    'kpp_shell': "kpp-sh",
    'neighbourhood_size': "nghb-1s",
    'neighbourhood_2_hop_size': "nghb-2s",
    'neighbourhood_size_discount': "nghb-sd",
    'page_rank': "p-rnk",
    'page_rank_mln': "p-rnk-m",
    'random': "random",
    'vote_rank': "v-rnk",
    'vote_rank_mln': "v-rnk-m",
}

nets_abbrv_map = {
    "arxiv": "arxiv",
    "timik1q2009": "timik",
}

In [ ]:
# for protocol OR we had budgets like [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30] 
# for protocol AND we had budgets like 
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
# but we'd like to skip wery low budgets where all methods are unaccurate and keep only:
# [15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] 
rows_to_drop = df.loc[
    (df["protocol"] == "AND") & 
    (
        ~df["seeding_budget"].isin(
            {15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40}
        )
    )
]
df = df.drop(rows_to_drop.index)

s_budgets = df["seeding_budget"].unique()
s_budgets_proto = {
    "OR": df.loc[df["protocol"] == "OR"]["seeding_budget"].unique(),
    "AND": df.loc[df["protocol"] == "AND"]["seeding_budget"].unique()
}
assert set(s_budgets) == set(s_budgets_proto["OR"]).union(set(s_budgets_proto["AND"]))

# drop unselected nets
rows_to_drop = df.loc[~df["network"].isin(set(nets))]
df = df.drop(rows_to_drop.index)


ss_methods, protocols, mi_vals, s_budgets, nets

## Convert data to a multidimensional matrices of obtained Gain

In [ ]:
cube = {}

for ssm in ss_methods:
    for net in nets:
        for proto in protocols:
            ddf = df.loc[
                (df["network"] == net) &
                (df["protocol"] == proto) &
                (df["selection_metric"] == ssm)
            ]
            ddf = pd.pivot_table(ddf, index="mi_value", columns="seeding_budget", values="gain").to_numpy()
            if len(ddf) == 0:
                raise ArithmeticError(f"Incorrect form of the dataframe for: {ssm}, {net}, {proto}")
            if not cube.get(ssm):
                cube[ssm] = {}
            if not cube[ssm].get(net):
                cube[ssm][net] = {}
            cube[ssm][net][proto] = ddf

len(cube)

In [ ]:
ssm_axis = []
nets_axis = []
protocols_axis = []

ssm_arrays = []

for ssm, net_proto_dict in cube.items():
    # print(ssm)
    ssm_axis.append(ssm)
    net_arrays = []

    for idx, (net, proto_dict) in enumerate(net_proto_dict.items()):
        # print(net)
        if len(nets_axis) == len(nets):
            assert nets_axis[idx] == net
        else:
            nets_axis.append(net)
        proto_arrs = []

        for idx, (proto, arr) in enumerate(proto_dict.items()):
            # print(proto)
            if len(protocols_axis) == len(protocols):
                assert protocols_axis[idx] == proto
            else:
                protocols_axis.append(proto)

            proto_arrs.append(arr)
        
        net_arrays.append(np.stack(proto_arrs))

    ssm_arrays.append(np.stack(net_arrays))

cube_gain = np.stack(ssm_arrays)

# array with all gains concatenated for all test runs - check shape (L==R)
cube_gain.shape, (len(ss_methods), len(nets), len(protocols), len(mi_vals), len(s_budgets_proto["AND"]))  # can be OR as well

In [ ]:
# we need to manually obtain indices of networks
print(nets_axis)

arxiv_idx = 0
timik1q2009_idx = 1

In [ ]:
gains_arxiv = cube_gain[:, arxiv_idx, ...]
gains_timik1q2009 = cube_gain[:, timik1q2009_idx, ...]

## Create effectiveness maps

In [ ]:
def grad(arr: np.ndarray, mi_ax: int, sb_ax: int) -> np.ndarray:
    arr_dmi = np.gradient(arr, axis=mi_ax)
    arr_dsb = np.gradient(arr, axis=sb_ax)
    arr_dmi_dsb = (np.sqrt((arr_dmi ** 2) + (arr_dsb ** 2)))[:, :, ::-1, :]
    return arr_dmi_dsb


def obtain_robust_idxs(arr: np.ndarray) -> List[Tuple[int, int]]:
    """Arr is a boolean matrix, output are coords."""
    robust_border_idxs = []
    for sb_idx, mi_tab in enumerate(arr.T):
        # print(mi_tab[::-1])
        for mi_idx, mi in enumerate(mi_tab[::-1]):
            if mi == True:
                robust_border_idxs.append([sb_idx, mi_idx])
                break
    return robust_border_idxs


def obtain_unrobust_idxs(arr: np.ndarray) -> List[Tuple[int, int]]:
    """Arr is a boolean matrix, output are coords."""
    unrobust_border_idxs = []
    for sb_idx, mi_tab in enumerate(arr.T):
        for mi_idx, mi in enumerate(mi_tab):
            if mi == True:
                unrobust_border_idxs.append([sb_idx, len(mi_tab) - 1 - mi_idx])
                break
    return unrobust_border_idxs


def convert_idxs(
        border_idxs: List[Tuple[int, int]], s_budgets_list: List[int]
    ) -> np.ndarray:
    robust_border_coords = [
        [s_budgets_list[sb_idx], mi_vals[mi_idx]] for 
        sb_idx, mi_idx in border_idxs
    ]
    robust_border_coords = np.array(robust_border_coords)
    return robust_border_coords


def obtain_robust_curve(arr: np.ndarray, s_budgets_list: List[int]) -> np.ndarray:
    return convert_idxs(obtain_robust_idxs(arr), s_budgets_list)


def obtain_unrobust_curve(arr: np.ndarray, s_budgets_list: List[int]) -> np.ndarray:
    return convert_idxs(obtain_unrobust_idxs(arr), s_budgets_list)


def log_func(x, a, b):
    return a * np.log(x) + b


def polynomial_3rd(x, a, b, c, d):
    return a * x ** 3 + b * x ** 2 + c * x + d


def sigmoid(x, a, b):
    return a * (np.exp(x) / (np.exp(x) + 1)) + b


def exp_func(x, a, b):
    return a * np.exp(x) + b


def optimize(func, curve_x, curve_y):
    parameters, covariance = curve_fit(func, curve_x, curve_y)
    std_err = np.sqrt(np.diag(covariance))
    fit_func = func(curve_x, *parameters)
    return (fit_func,
            {idx: val for idx, val in enumerate(std_err)},
            {idx: val for idx, val in enumerate(parameters)},
    )

In [ ]:
# select arxiv network
analysed_arr = gains_arxiv
analysed_net = nets[arxiv_idx]

# # or timik
# analysed_arr = gains_timik1q2009
# analysed_net = nets[timik1q2009_idx]

# select function to aproximate curves with
func = log_func

print(analysed_net, analysed_arr.shape)

In [ ]:
out_dir = Path("_results/top_efficiency_maps")
out_dir.mkdir(exist_ok=True, parents=True)

vis_name = out_dir / f"map_net-{analysed_net}.pdf"
fit_err_name = out_dir / f"params-err_net-{analysed_net}.csv"
fit_par_name = out_dir / f"params-fit_net-{analysed_net}.csv"
fit_bulk_err_name = out_dir / f"params-err-avg_net-{analysed_net}.csv"

In [ ]:
grad_arr = grad(analysed_arr, mi_ax=-2, sb_ax=-1)

curves = {}

for idx, ssm_proto_grad in enumerate(grad_arr):
    method = ss_methods_abbrv_map[ss_methods[idx]]
    # print(method, ssm_proto_grad.shape)

    protos = {}

    for iidx, proto_grad in enumerate(ssm_proto_grad):
        proto = protocols[iidx]
        # print(proto, proto_grad.shape)

        # sns.heatmap(
        #     proto_grad,
        #     xticklabels=s_budgets_proto[proto],
        #     yticklabels=mi_vals[::-1],
        #     cbar=False,
        #     annot=True,
        #     annot_kws={"size": 7},
        # )
        # plt.title(f"raw gradinent for {method} and {proto}")
        # plt.tight_layout()
        # plt.savefig(out_dir / f"grad_{method}_{proto}_raw.pdf")
        # plt.close()

        proto_grad = proto_grad > 10

        # sns.heatmap(
        #     proto_grad,
        #     xticklabels=s_budgets_proto[proto],
        #     yticklabels=mi_vals[::-1],
        #     cbar=False,
        #     annot=True,
        #     annot_kws={"size": 7},
        # )
        # plt.title(f"thresholded gradinent for {method} and {proto}")
        # plt.tight_layout()
        # plt.savefig(out_dir / f"grad_{method}_{proto}_thre.pdf")
        # plt.close()

        protos[proto] = {
            "effective": obtain_robust_curve(proto_grad, s_budgets_proto[proto]),
            "ineffective": obtain_unrobust_curve(proto_grad, s_budgets_proto[proto])
        }

    curves[method] = protos

In [ ]:
# a particular visualisation on curve fitting

_method  = "v-rnk-m"
_proto = "OR"


grad_robust = curves[_method][_proto]["effective"]
grad_unrobust = curves[_method][_proto]["ineffective"]

pts = np.linspace(1, max(s_budgets_proto["OR"]))

fit_func_robust, std_err_robust, par_robust = optimize(log_func, grad_robust[:, 0], grad_robust[:, 1])
fit_func_robust = log_func(pts, *par_robust.values())

fit_func_unrobust, std_err_unrobust, par_unrobust = optimize(log_func, grad_unrobust[:, 0], grad_unrobust[:, 1])
fit_func_unrobust = log_func(pts, *par_unrobust.values()) 

plt.ylim(min(mi_vals), max(mi_vals))
plt.xlim(min(pts), max(pts))

plt.fill_between(pts, fit_func_robust, alpha=.5, color="green")
plt.fill_between(pts, fit_func_unrobust, max(mi_vals) * len(fit_func_unrobust), alpha=.5, color="red")
plt.fill_between(pts, fit_func_unrobust, fit_func_robust, alpha=.5, color="orange")

plt.title(f"map for {_method} with {_proto} on {analysed_net}")
plt.tight_layout()
plt.show()
# plt.savefig(out_dir / f"map_{_method}.png")
# plt.close()

print("Robust coeff: ", par_robust, "std_err:", std_err_robust)
print("Unobust coeff: ", par_unrobust, "std_err:", std_err_unrobust)

In [ ]:
color_iter = iter(plt.cm.jet(np.linspace(0, 1, len(ss_methods))))
colors_ssm = {ss_methods_abbrv_map[ssm]: next(color_iter) for ssm in ss_methods}
line_width = 1.5

fit_errors = {}
fit_params = {}

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), gridspec_kw={"width_ratios": [50, 50]})
fig.tight_layout(rect=(0.05, 0.05, 0.95, 0.95))
title = f"Efficiency curves on {analysed_net}; protocols from left: {protocols[0]}, {protocols[1]}"


for ssm, proto_curves in curves.items():
    # print(ssm)

    for proto, curve in proto_curves.items():
        # print(proto)

        pts_robust = curve["effective"]
        pts_unrobust = curve["ineffective"]
        s_budgets_pts = np.linspace(1, max(s_budgets_proto[proto]))

        fit_func_robust, std_err_robust, par_robust = optimize(func, pts_robust[:, 0], pts_robust[:, 1])
        fit_func_robust = func(s_budgets_pts, *par_robust.values())

        fit_func_unrobust, std_err_unrobust, par_unrobust = optimize(func, pts_unrobust[:, 0], pts_unrobust[:, 1])
        fit_func_unrobust = func(s_budgets_pts, *par_unrobust.values())

        fit_errors[(ssm, proto, "effective")] = std_err_robust
        fit_errors[(ssm, proto, "ineffective")] = std_err_unrobust
        fit_params[(ssm, proto, "effective")] = par_robust
        fit_params[(ssm, proto, "ineffective")] = par_unrobust

        idx = protocols.index(proto)
        linestyle=(4 * random.random(), (4, 1))
        ax[idx].plot(
            s_budgets_pts,
            fit_func_robust,
            label=ssm,
            marker='',
            color=colors_ssm[ssm],
            linewidth=line_width,
            linestyle=linestyle,
            alpha=.75,
        )

for axx, proto in zip(ax, protocols):
    axx.tick_params(axis='both', which='major', labelsize=8)
    axx.set_xlim(min(s_budgets_proto[proto]), max(s_budgets_proto[proto]))
    axx.set_xticks(s_budgets_proto[proto])
    axx.set_ylim(min(mi_vals), max(mi_vals))
    axx.legend(loc="lower right")
    axx.grid()
    axx.set_xlabel("seeding_budget")
    axx.set_ylabel("mi_value")

fig.suptitle(title)
fig.savefig(vis_name, dpi=300)
plt.close(fig)

In [ ]:
fit_errors_df = pd.DataFrame(fit_errors).T
fit_errors_df.index.names = ["method", "protocol", "curve_type"]
# fit_errors_df.to_csv(fit_err_name)

In [ ]:
fit_params_df = pd.DataFrame(fit_params).T
fit_params_df.index.names = ["method", "protocol", "curve_type"]
# fit_params_df.to_csv(fit_par_name)

## Compute avg error of curve fitting

In [ ]:
_fit_errors_df = fit_errors_df.reset_index()
unrobust_to_drop = _fit_errors_df.loc[(_fit_errors_df["curve_type"] == "ineffective")].index
_fit_errors_df = _fit_errors_df.drop(unrobust_to_drop, axis=0)

In [ ]:
_fit_params_df = fit_params_df.reset_index()
unrobust_to_drop = _fit_params_df.loc[(_fit_params_df["curve_type"] == "ineffective")].index
_fit_params_df = _fit_params_df.drop(unrobust_to_drop, axis=0)

In [ ]:
fit_params_errors_df = pd.merge(_fit_errors_df, _fit_params_df, on=["method", "curve_type", "protocol"], suffixes=["_err", "_par"])
fit_params_errors_df["0_prct_err"] = np.abs(100 * fit_params_errors_df["0_err"] / fit_params_errors_df["0_par"])
fit_params_errors_df["1_prct_err"] = np.abs(100 * fit_params_errors_df["1_err"] / fit_params_errors_df["1_par"])
# fit_params_errors_df["2_prct_err"] = np.abs(100 * fit_params_errors_df["2_err"] / fit_params_errors_df["2_par"])
# fit_params_errors_df["3_prct_err"] = np.abs(100 * fit_params_errors_df["3_err"] / fit_params_errors_df["3_par"])
fit_params_errors_df

In [ ]:
mean_prct_err_df = fit_params_errors_df[["0_prct_err", "1_prct_err", "protocol"]].groupby("protocol").mean()
# mean_prct_err_df = fit_params_errors_df[["0_prct_err", "1_prct_err", "2_prct_err", "3_prct_err", "protocol"]].groupby("protocol").mean()
mean_prct_err_df

In [ ]:
fit_params_errors_df.to_csv(fit_par_name)
mean_prct_err_df.to_csv(fit_bulk_err_name)